In [ ]:
# Remote Training on SSH with CUDA

This notebook lets you launch training on an SSH machine with CUDA and safely disconnect while training continues.

**How it works:**
- Cell 2: Launches training as a background process with `nohup` (survives SSH disconnect)
- Cell 3: Monitor recent training logs
- Cell 4: Check GPU utilization
- Cell 5: Stop training if needed

**Workflow:**
1. Run Cell 2 to start training
2. Disconnect from SSH/close notebook - training continues!
3. Reconnect later and run Cell 3 to check progress
4. Check outputs in `checkpoints/` and `outputs/` directories when done

In [5]:
import os
from datetime import datetime

# Create logs directory if it doesn't exist
os.makedirs('logs', exist_ok=True)

# Generate unique log filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_file = f"logs/training_{timestamp}.log"

print(f"Starting training in background...")
print(f"Logs will be written to: {log_file}")
print(f"To monitor progress: tail -f {log_file}")
print(f"To check if still running: ps aux | grep 'python main.py'")

# Launch training as detached background process
# nohup ensures it continues after SSH disconnect
# &> redirects both stdout and stderr to log file
# & runs in background
# The process will continue even if you close this notebook or disconnect SSH
cmd = (
    f"nohup python main.py "
#    f"-m training.loss.adaptive.strategy=fixed,equal_init,equal_init_ema,ema "
#    f"training.loss.use_pi=true,false "
    f"-m training.loss.use_rk4=false,true "
#    f"training.loss.use_energy=true,false "
    f"&> {log_file} &"
)
os.system(cmd)

print("\n✓ Training launched in background!")
print(f"\nUseful commands:")
print(f"  Monitor live:     tail -f {log_file}")
print(f"  Check status:     ps aux | grep 'python main.py'")
print(f"  Kill if needed:   pkill -f 'python main.py'")
print(f"  GPU usage:        nvidia-smi")

Starting training in background...
Logs will be written to: logs/training_20251030_091605.log
To monitor progress: tail -f logs/training_20251030_091605.log
To check if still running: ps aux | grep 'python main.py'

✓ Training launched in background!

Useful commands:
  Monitor live:     tail -f logs/training_20251030_091605.log
  Check status:     ps aux | grep 'python main.py'
  Kill if needed:   pkill -f 'python main.py'
  GPU usage:        nvidia-smi
[2025-10-30 09:16:07,625][HYDRA] Launching 2 jobs locally
[2025-10-30 09:16:07,625][HYDRA] 	#0 : training.loss.use_rk4=False
[2025-10-30 09:16:07,724][__main__][INFO] - ==================================================
[2025-10-30 09:16:07,724][__main__][INFO] - CONFIGURATION
[2025-10-30 09:16:07,724][__main__][INFO] - ==================================================
[2025-10-30 09:16:07,727][__main__][INFO] - 
dataset:
  num_graphs: 100
  num_steps: 6
  dt: 0.01
  train_ratio: 0.8
  batch_size: 4
  shuffle: true
  drop_last: false


Error executing job with overrides: ['training.loss.use_rk4=False']
Traceback (most recent call last):
  File "/home/jupyter-gcap/wave_gnn/main.py", line 228, in main
    model, metrics = train_model(
  File "/home/jupyter-gcap/wave_gnn/train.py", line 555, in train_model
    checkpoint = torch.load(save_path, map_location=device)
  File "/home/jupyter-gcap/.local/lib/python3.10/site-packages/torch/serialization.py", line 1529, in load
    raise pickle.UnpicklingError(_get_wo_message(str(e))) from None
_pickle.UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, do those steps only if you trust the source of the checkpoint. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternat

In [ ]:
# Monitor the most recent training log
import glob
import time

# Find most recent log file
log_files = sorted(glob.glob('logs/training_*.log'))
if log_files:
    latest_log = log_files[-1]
    print(f"Monitoring: {latest_log}\n")
    print("="*70)
    
    # Display last 30 lines
    with open(latest_log, 'r') as f:
        lines = f.readlines()
        for line in lines[-30:]:
            print(line.rstrip())
else:
    print("No training logs found yet.")

In [ ]:
# Check GPU utilization
import subprocess

result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout)

In [ ]:
# Stop training if needed (kills all python main.py processes)
import subprocess

print("Stopping all training processes...")
subprocess.run(['pkill', '-f', 'python main.py'])
print("✓ Training processes stopped")

# Verify
result = subprocess.run(['pgrep', '-f', 'python main.py'], capture_output=True)
if result.stdout:
    print("⚠ Some processes still running, try again or use: kill -9 <PID>")
else:
    print("✓ All training processes terminated")